## Utilizando o modelo MLP baseado no MNist para inferir

In [1]:
%maven ai.djl:api:0.28.0
%maven ai.djl:model-zoo:0.28.0
%maven ai.djl.mxnet:mxnet-engine:0.28.0
%maven ai.djl.mxnet:mxnet-model-zoo:0.28.0
%maven org.slf4j:slf4j-simple:2.0.13

import java.awt.image.*;
import java.nio.file.*;
import java.util.*;
import java.util.stream.*;
import ai.djl.*;
import ai.djl.basicmodelzoo.basic.*;
import ai.djl.ndarray.*;
import ai.djl.modality.*;
import ai.djl.modality.cv.*;
import ai.djl.modality.cv.util.NDImageUtils;
import ai.djl.translate.*;

### Buscando imagem na pasta Data com algumas imagens

In [2]:
var img = ImageFactory.getInstance().fromInputStream(new FileInputStream("../data/mnist/5.png"));
img.getWrappedImage();

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


### Carregando o modelo previamente gerado

In [3]:
Path modelDir = Paths.get("buildModel/mlpNotebook");
Model model = Model.newInstance("mlp");
model.setBlock(new Mlp(28 * 28, 10, new int[] {128, 64}));
model.load(modelDir);

### Produzindo o Translator

In [4]:
Translator<Image, Classifications> translator = new Translator<Image, Classifications>() {

    @Override
    public NDList processInput(TranslatorContext ctx, Image input) {
        // Convert Image to NDArray
        NDArray array = input.toNDArray(ctx.getNDManager(), Image.Flag.GRAYSCALE);
        return new NDList(NDImageUtils.toTensor(array));
    }

    @Override
    public Classifications processOutput(TranslatorContext ctx, NDList list) {
        // Create a Classifications with the output probabilities
        NDArray probabilities = list.singletonOrThrow().softmax(0);
        List<String> classNames = IntStream.range(0, 10).mapToObj(String::valueOf).collect(Collectors.toList());
        return new Classifications(classNames, probabilities);
    }

    @Override
    public Batchifier getBatchifier() {
        // The Batchifier describes how to combine a batch together
        // Stacking, the most common batchifier, takes N [X1, X2, ...] arrays to a single [N, X1, X2, ...] array
        return Batchifier.STACK;
    }
};

### Criar o Predictor, que é a classe principal que orquestrará o processo de inferência

In [5]:
var predictor = model.newPredictor(translator);

### Executar a inferência

In [6]:
var classifications = predictor.predict(img);

classifications

[
	{"class": "5", "probability": 0,99491}
	{"class": "0", "probability": 0,00271}
	{"class": "6", "probability": 0,00090}
	{"class": "3", "probability": 0,00070}
	{"class": "8", "probability": 0,00060}
]